In [1]:
library(R.matlab)
library(pracma)
library(parallel)
mcc <- 20

R.matlab v3.6.1 (2016-10-19) successfully loaded. See ?R.matlab for help.

Attaching package: ‘R.matlab’

The following objects are masked from ‘package:base’:

    getOption, isOpen



In [2]:
fcs1 = readMat(gzfile("/data/SharedData/fingerprint_data/FL_Matrices/All_Sub_REST1_FL.mat.gz"))

In [3]:
fcs2 = readMat(gzfile("/data/SharedData/fingerprint_data/FL_Matrices/All_Sub_REST2_FL.mat.gz"))

In [4]:
dim(fcs1[[1]])

[1] 338 268 268

In [5]:
dim(fcs2[[1]])

[1] 338 268 268

In [ ]:
fc_flatten <- function(arr) {
  apply(arr, 1, function(a) {
    a[upper.tri(a)]
  })
}


nsubs <- nrow(fcs1[[1]])
## shuffle subjects
shufind <- sample(nsubs, nsubs, replace = FALSE)

fcz1 <- fc_flatten(fcs1[[1]])
fcz2 <- fc_flatten(fcs2[[1]])

In [ ]:
r12 <- cor(fcz1, fcz2)

In [ ]:
mats1 <- fcs1[[1]]
mats2 <- fcs2[[1]]

In [ ]:
nsubs1 <- 338
nsubs2 <- 338

ijs <- apply(cbind(rep(1:nsubs1, each=nsubs2), rep(1:nsubs2, nsubs1)), 1, list)
get_kl_subroutine <- function(ij) {
  i <- ij[[1]][1]
  j <- ij[[1]][2]
  vstuff <- solve(mats1[i,,], mats2[j,,])
  vstuff2 <- solve(mats2[i,,], mats1[j,,])  
  pmin(sum(diag(vstuff)) - log(det(vstuff)), sum(diag(vstuff2)) - log(det(vstuff2)))
}

get_kl_subroutine1 <- function(ij) {
  i <- ij[[1]][1]
  j <- ij[[1]][2]
  vstuff <- solve(mats1[i,,], mats2[j,,])
  sum(diag(vstuff)) - log(det(vstuff))
}

t1 <- proc.time()
kls <- mclapply(ijs, get_kl_subroutine, mc.cores = mcc)
proc.time() - t1

klds <- t(matrix(unlist(kls), nsubs2, nsubs1))

In [ ]:
dim(klds)

In [ ]:
write.csv(klds, 'FL_noGSR_kl.csv', row.names = FALSE)

In [ ]:
write.csv(r12, 'FL_noGSR_cor.csv', row.names = FALSE)